<a href="https://colab.research.google.com/github/pinilDissanayaka/Multi-Vector-Retriever-for-RAG-on-Tables-Texts/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt install tesseract-ocr -y
!sudo apt install libtesseract-dev -y
!sudo apt-get install poppler-utils -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (5,372 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [2]:
!pip install langchain unstructured[all-docs] pydantic lxml openai chromadb tiktoken opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

In [ ]:
pip install langchain-google-vertexai --upgrade

In [ ]:
pip install --upgrade nltk

In [113]:
import os
import uuid
import requests
import base64
from unstructured.partition.pdf import partition_pdf
from langchain_core.documents import Document
from langchain.prompts import PromptTemplate
from langchain_groq.chat_models import ChatGroq
from langchain_google_vertexai import ChatVertexAI
from langchain.embeddings import VertexAIEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.schema.document import Document
from langchain.vectorstores import FAISS
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from google.colab import userdata

In [33]:
os.environ['GROQ_API_KEY']=userdata.get('GROQ_API_KEY')
os.environ["GOOGLE_API_KEY"]=userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_PROJECT_ID"]=userdata.get('GOOGLE_PROJECT_ID')

In [77]:
file_path="/content/llama3.pdf"

with open(file_path, "wb") as file:
  response = requests.get("https://arxiv.org/pdf/2302.13971")
  file.write(response.content)

In [7]:
output_path = "./images"

In [84]:
raw_pdf_elements = partition_pdf(
    filename=file_path,
    extract_images_in_pdf=True,
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    extract_image_block_output_dir=output_path,
)

In [85]:
for element in raw_pdf_elements:
  print(type(element))

<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.Table'>
<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.Table'>
<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.Table'>
<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.Table'>
<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.Table'>
<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.Table'>
<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.Table'>
<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstruc

In [86]:
llm=ChatGroq(model="llama3-groq-70b-8192-tool-use-preview", temperature=0.2)

In [87]:
summary_prompt_template='''Summarize the key insights from the table.
Focus on highlighting major trends, patterns, or significant values.
Include comparisons where applicable and note any outliers or anomalies.
Provide a clear and concise summary that captures the essence of the {table_data}.'''


summary_prompt=PromptTemplate.from_template(summary_prompt_template)

print(f"Summary prompt {summary_prompt}")

summary_chain=summary_prompt | llm | StrOutputParser()

Summary prompt input_variables=['table_data'] template='Summarize the key insights from the table.\nFocus on highlighting major trends, patterns, or significant values.\nInclude comparisons where applicable and note any outliers or anomalies.\nProvide a clear and concise summary that captures the essence of the {table_data}.'


In [107]:
table_count=0
table_elements=list()
table_summary=list()

text_count=0
text=list()

for element in raw_pdf_elements:
  if str(type(element))=="<class 'unstructured.documents.elements.Table'>":
    table_count+=1
    table_elements.append(element.text)
    table_summary.append(Document(
        page_content=summary_chain.invoke({"table_data" :element.text }),
        metadata={
            "id":str(uuid.uuid4()),
            "type":"table"
        }
        )
    )
  else:
    text_count+=1
    text.append(Document(
        page_content=element.text,
        metadata={
            "id":str(uuid.uuid4()),
            "type":"text"
        }
        )
    )

In [108]:
print(f"Text element count = {text_count}")
print(f"Table element count = {table_count}")

Text element count = 42
Table element count = 16


In [109]:
gemini= ChatVertexAI(
    model="gemini-1.5-pro",
    temperature=0.2,
    max_tokens=None,
    max_retries=6,
    stop=None,
)

In [110]:
image_summary_prompt_template='''Summarize the key insights from the image.
Focus on highlighting major trends, patterns, or significant values.
Include comparisons where applicable and note any outliers or anomalies.
Provide a clear and concise summary that captures the essence of the {image_data}.'''


image_summary_prompt=PromptTemplate.from_template(image_summary_prompt_template)

print(f"Summary prompt {image_summary_prompt}")

image_summary_chain=image_summary_prompt | gemini | StrOutputParser()

Summary prompt input_variables=['image_data'] template='Summarize the key insights from the image.\nFocus on highlighting major trends, patterns, or significant values.\nInclude comparisons where applicable and note any outliers or anomalies.\nProvide a clear and concise summary that captures the essence of the {image_data}.'


In [112]:
image_count=0
image_elements=list()
image_summary=[]


for image in os.listdir(output_path):
  image_path=os.path.join(output_path, image)
  with open(image_path, "rb") as img:
    image_data=img.read()
    base64_encoded_data=base64.b64encode(image_data).decode('utf-8')
    print(len(base64_encoded_data))
  image_count+=1
  image_elements.append(base64_encoded_data)
  image_summary.append(
      Document(
          page_content=image_summary_chain.invoke({"image_data":base64_encoded_data}),
          metadata={
              "id":str(uuid.uuid4()),
              "type":"image",
              "image_encode":base64_encoded_data
              }
          )
      )



208052
62084


In [114]:
documents=[text, table_summary, image_summary]

In [115]:
embeddings=GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")